## Idea Trabajo Práctico Final: Predicción de Resultado de Partidos de Tenis 
El objetivo de la red neuronal a implementar es poder predecir los resultados de cierta temporada de tenis actual considerando el historial de partidos de temporadas pasadas, como así también el historial individual entre los jugadores de cada partido. 
### Dataset de Referencia
El dataset de referencia será el historial de partidos de la Asociación de Tenistas Profesionales (ATP) brindado en su base de datos. Este no sólo incluye quien ha ganado cada partido, sino el puntaje y los porcentajes de las características relevantes de cada jugada en particular ('primeros servicios', 'segundos servicios', 'aces', 'porcentaje de devoluciones', 'doble faltas', etc.).
Por lo tanto, el set de entrenamiento de la red consistirá de un vector de características de entrada (X) representando las características de los jugadores involucrados y del partido, y un vector de salida correspondiente que determina que jugador gana. 
A continuación se realiza el parseo de datos:


In [2]:
import numpy as np
import pandas as pd
import re
import numpy


from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

#test = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')
#pd_atp_matches_2017.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]
#pd.set_option('display.max_rows', 5000)
#pd_atp_matches_2017
#pd_atp_matches_2017.iloc[1]

In [9]:
pd_atp_matches_2017 = pd.read_csv('tennis_atp/atp_matches_2010_2016.csv')
my_data_stats = pd.read_csv('data/match_stats_2010_2016.csv')

#select what do I want from the stats dataset
mycolumns=[
           'ms_winner_aces','ms_loser_aces','ms_winner_double_faults','ms_loser_double_faults',
           'ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serves_in',
           'ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serves_in',
           'ms_winner_return_points_won','ms_winner_return_points_total',
           'ms_loser_return_points_won','ms_loser_return_points_total',
           'ms_winner_total_points_won','ms_winner_total_points_total',
            'ms_winner_second_serve_points_total','ms_loser_second_serve_points_total']
new_columns=[
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
            'w_2ndTot','l_2ndTot']

my_data_stats=my_data_stats.filter(mycolumns, axis=1)
my_data_stats.columns = new_columns

#select what I want from the other dataset
labels=['tourney_id','tourney_name','round','winner_id','winner_name','loser_id','loser_name','score',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points',
        'w_ace','l_ace','w_df','l_df',
        'w_svpt','w_1stWon','w_1stIn',
        'l_svpt','l_1stWon','l_1stIn',
        'w_2ndWon','l_2ndWon',
        'w_bpSaved','w_bpFaced',
        'l_bpSaved','l_bpFaced']
pd_atp_matches_2017=pd_atp_matches_2017.filter(labels, axis=1)

labels_match=['w_ace','l_ace','w_df','l_df',
                'w_svpt','w_1stWon','w_1stIn',
                'l_svpt','l_1stWon','l_1stIn']
pd_atp_matches_2017=pd.merge(pd_atp_matches_2017,my_data_stats,on=labels_match).reset_index(drop=True)
pd_atp_matches_2017=pd_atp_matches_2017.drop_duplicates(keep='first').reset_index(drop=True)

WRP1=pd.read_csv('WRP1t.csv')
pd_atp_matches_2017['WRP1'] = WRP1['WRP1']

WRP2=pd.read_csv('WRP2t.csv')
pd_atp_matches_2017['WRP2'] = WRP2['WRP2']

WRP=pd.read_csv('WRPt.csv')
pd_atp_matches_2017['WRP'] = WRP['WRP']

TMW=pd.read_csv('TMWt.csv')
pd_atp_matches_2017['TMW'] = TMW['TMW']


#calculate number of games
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

pd_atp_matches_2017=pd_atp_matches_2017.assign(TotGames=0)
for i in range(0,len(pd_atp_matches_2017.index)):
    str1=pd_atp_matches_2017.loc[i, 'score']
    
    start=findOccurrences(str1, '(')
    end=findOccurrences(str1, ')')

    for k in range(0,len(start)):
        for l in range(0,end[k]-start[k]+1):
            str1=change_char(str1,start[k]+l,' ')

    number_of_games=0
    for j in range (0,len(list(filter(str.isdigit, str1)))):
        number_of_games=number_of_games+int(list(filter(str.isdigit, str1))[j])
    pd_atp_matches_2017.loc[i, 'TotGames'] = number_of_games

    
    
def SetNan(pandas,column):
    list = pandas.index[(pandas[column].isin([0])) == True].tolist()
    for i in range(0,len(list)):
        pandas.loc[list[i],column]=np.nan
        
SetNan(pd_atp_matches_2017,'w_svpt')
SetNan(pd_atp_matches_2017,'l_svpt')
SetNan(pd_atp_matches_2017,'w_1stIn')
SetNan(pd_atp_matches_2017,'l_1stIn')
SetNan(pd_atp_matches_2017,'w_2ndTot')
SetNan(pd_atp_matches_2017,'l_2ndTot')

Debajo se muestra un ejemplo del dataset scores obtenido:

In [20]:
pd_atp_matches_2017

,tourney_id,tourney_name,round,winner_id,winner_name,loser_id,loser_name,score,winner_rank,winner_rank_points,...,l_ret_tot,w_pts_w,total_points,w_2ndTot,l_2ndTot,WRP1,WRP2,WRP,TMW,TotGames
0,2013-319,Kitzbuhel,R32,104919,Leonardo Mayer,104932,Kenny De Schepper,6-4 6-4,78.0,641.0,...,54.0,73.0,132.0,21.0,33.0,0.322187,0.305694,0.016493,1.000000,20
1,2013-319,Kitzbuhel,R32,104547,Horacio Zeballos,104770,Martin Fischer,7-5 1-6 6-3,54.0,862.0,...,98.0,89.0,179.0,27.0,37.0,0.319960,0.332126,-0.012166,1.000000,28
2,2013-319,Kitzbuhel,R32,104719,Marcel Granollers,106207,Mate Pavic,7-6(2) 6-2,53.0,870.0,...,83.0,84.0,156.0,34.0,34.0,0.367611,0.326035,0.041576,1.000000,21
3,2013-319,Kitzbuhel,R32,104198,Guillermo Garcia Lopez,105064,Thomaz Bellucci,6-7(1) 6-3 7-6(4),72.0,686.0,...,132.0,126.0,248.0,67.0,54.0,0.377084,0.372715,0.004369,0.500000,35
4,2013-319,Kitzbuhel,R32,104898,Robin Haase,105558,Guillaume Rufin,6-4 7-6(4),68.0,717.0,...,81.0,90.0,169.0,30.0,40.0,0.350511,0.329149,0.021362,1.000000,23
5,2013-319,Kitzbuhel,R32,104978,Daniel Brands,104122,Carlos Berlocq,7-6(4) 2-6 6-2,52.0,873.0,...,84.0,80.0,166.0,41.0,20.0,0.301639,0.371560,-0.069921,1.000000,29
6,2013-319,Kitzbuhel,R32,103656,Albert Montanes,104494,Adrian Ungur,6-4 6-2,45.0,910.0,...,52.0,64.0,111.0,15.0,24.0,0.346747,0.313839,0.032909,1.000000,18
7,2013-319,Kitzbuhel,R32,103812,Victor Hanescu,105379,Aljaz Bedene,6-2 6-1,46.0,909.0,...,41.0,55.0,84.0,7.0,25.0,0.345751,0.348477,-0.002726,1.000000,15
8,2013-319,Kitzbuhel,R32,106233,Dominic Thiem,105723,Andrey Kuznetsov,6-2 7-5,268.0,167.0,...,69.0,68.0,125.0,33.0,25.0,0.354054,0.365435,-0.011380,1.000000,20
9,2013-319,Kitzbuhel,R32,104225,Jan Hajek,105060,Aldin Setkic,6-2 6-1,135.0,430.0,...,41.0,56.0,88.0,15.0,19.0,0.426538,NaN,NaN,1.000000,15


## Extracción de Features 
A continuación se describiran la extracción de features de cada partido. 

A continuación se busca la lista de todos los jugadores presentes.

In [17]:
all_winner_players=pd_atp_matches_2017.filter(['winner_id','winner_name'], axis=1)
all_winner_players.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)

all_loser_players=pd_atp_matches_2017.filter(['loser_id','loser_name'], axis=1)
all_loser_players.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)

#print(all_winner_players)
#print(all_loser_players)

all_players=all_winner_players.append(all_loser_players, ignore_index=True).drop_duplicates()

all_players=all_players.sort_values(by=['ID']).reset_index(drop=True);
#players_list=all_players.ID.unique()
#players_list.sort()
#players_list
#all_players

Luego, se determina el feature que indica cual jugador tuvo mejor historial en los partidos anteriores.

In [18]:
#this function returns WRP of a player against a certain adversary

def getMatchesWonPer(myopponent,adversary):
    exp1=pd_atp_matches_2017['winner_id'].isin([myopponent])
    exp2=pd_atp_matches_2017['loser_id'].isin([adversary])
    exp3=pd_atp_matches_2017['loser_id'].isin([myopponent])
    exp4=pd_atp_matches_2017['winner_id'].isin([adversary])

    matches_won=len(pd_atp_matches_2017.loc[(exp1&exp2)].index)
    matches_lost=len(pd_atp_matches_2017.loc[(exp3&exp4)].index)
    matchesWonPer = matches_won/(matches_won+matches_lost)
    
    return matchesWonPer

pd_atp_matches_2017=pd_atp_matches_2017.assign(TMW=0)
for i in range(0,len(pd_atp_matches_2017.index)):
    TMW1=getMatchesWonPer(pd_atp_matches_2017.loc[i, 'winner_id'],pd_atp_matches_2017.loc[i, 'loser_id'])
    pd_atp_matches_2017.loc[i, 'TMW'] = TMW1 - (1-TMW1)
    
pd_atp_matches_2017['TMW'].to_csv('TMWt.csv',header='false',index='false')

Se buscan los oponentes en común y se calcula el promedio del porcentaje de puntos devueltos ganados.

In [19]:
#this function returns WRP of a player against a certain adversary

def getWRPi(myopponent,adversary):
    exp1=pd_atp_matches_2017['winner_id'].isin([myopponent])
    exp2=pd_atp_matches_2017['loser_id'].isin([adversary])
    exp3=pd_atp_matches_2017['loser_id'].isin([myopponent])
    exp4=pd_atp_matches_2017['winner_id'].isin([adversary])

    matches_won=pd_atp_matches_2017.loc[(exp1&exp2)]
    matches_won=matches_won.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','w_ret_w','w_ret_tot','l_ret_w','l_ret_tot'], axis=1)

    matches_lost=pd_atp_matches_2017.loc[(exp3&exp4)]
    matches_lost=matches_lost.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','w_ret_w','w_ret_tot','l_ret_w','l_ret_tot'], axis=1)

    #calculate WRP (winning return percentage) of all the matches against the adversary
    matches_won['WRP'] = matches_won['w_ret_w']/matches_won['w_ret_tot']
    matches_lost['WRP'] = matches_lost['l_ret_w']/matches_lost['l_ret_tot']
    matches_won=matches_won.append(matches_lost)
    matches_won
    return matches_won['WRP'].mean()


def getWRP(id1,id2):

    #id1 = 105223 #delpo
    #id2 = 105992 #Ryan harrison

    #id1 = 104229 #nadal
    #id2 = 105032 #groth
    array=[id1,id2]
    locate_1=all_players.loc[(all_players['ID'].isin([array[0]]))]
    locate_2=all_players.loc[(all_players['ID'].isin([array[1]]))]

    #print('El jugador #1 es: ' + locate_1.iloc[0,1])
    #print('El jugador #2 es: ' + locate_2.iloc[0,1])

    #print('En el 2017 se enfrentaron a: ' )

    opponents_p1_1=pd_atp_matches_2017.loc[(pd_atp_matches_2017['winner_id'].isin([array[0]]))]
    opponents_p1_1=opponents_p1_1[['loser_id','loser_name']]
    opponents_p1_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p1_2=pd_atp_matches_2017.loc[(pd_atp_matches_2017['loser_id'].isin([array[0]]))]
    opponents_p1_2=opponents_p1_2[['winner_id','winner_name']]
    opponents_p1_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p1=opponents_p1_1.append(opponents_p1_2, ignore_index=True).drop_duplicates()

    opponents_p2_1=pd_atp_matches_2017.loc[(pd_atp_matches_2017['winner_id'].isin([array[1]]))]
    opponents_p2_1=opponents_p2_1[['loser_id','loser_name']]
    opponents_p2_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p2_2=pd_atp_matches_2017.loc[(pd_atp_matches_2017['loser_id'].isin([array[1]]))]
    opponents_p2_2=opponents_p2_2[['winner_id','winner_name']]
    opponents_p2_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p2=opponents_p2_1.append(opponents_p2_2, ignore_index=True).drop_duplicates()

    common_opponents = pd.merge(opponents_p1, opponents_p2, how='inner', on=['ID']).filter(['ID','NAME_x'], axis=1)
    common_opponents=common_opponents.assign(WRP1=0,WRP2=0)

    for i in range(0,len(common_opponents.index)):
        common_opponents.loc[i, 'WRP1'] = getWRPi(id1,common_opponents.loc[i, 'ID'])
        common_opponents.loc[i, 'WRP2'] = getWRPi(id2,common_opponents.loc[i, 'ID'])

    WRP1=common_opponents['WRP1'].mean()
    WRP2=common_opponents['WRP2'].mean()
    #WRP=WRP1-WRP2
    #print(WRP)

    #display_side_by_side(opponents_p1,opponents_p2,common_opponents)
    return(WRP1,WRP2)

##Run to get WRP

pd_atp_matches_2017=pd_atp_matches_2017.assign(WRP=0)

for i in range(0,len(pd_atp_matches_2017.index)):
    WRP1,WRP2 = getWRP(pd_atp_matches_2017.loc[i, 'winner_id'],pd_atp_matches_2017.loc[i, 'loser_id'])
    pd_atp_matches_2017.loc[i, 'WRP1'] = WRP1
    pd_atp_matches_2017.loc[i, 'WRP2'] = WRP2
    pd_atp_matches_2017.loc[i, 'WRP'] = WRP1-WRP2
    print('Processing match #' + str(i) + '\n')
    
pd_atp_matches_2017['WRP1'].to_csv('WRP1t.csv',header='false',index='false')
pd_atp_matches_2017['WRP2'].to_csv('WRP2t.csv',header='false',index='false')
pd_atp_matches_2017['WRP'].to_csv('WRPt.csv',header='false',index='false')

Processing match #0

Processing match #1

Processing match #2

Processing match #3

Processing match #4

Processing match #5

Processing match #6

Processing match #7

Processing match #8

Processing match #9

Processing match #10

Processing match #11

Processing match #12

Processing match #13

Processing match #14

Processing match #15

Processing match #16

Processing match #17

Processing match #18

Processing match #19

Processing match #20

Processing match #21

Processing match #22

Processing match #23

Processing match #24

Processing match #25

Processing match #26

Processing match #27

Processing match #28

Processing match #29

Processing match #30

Processing match #31

Processing match #32

Processing match #33

Processing match #34

Processing match #35

Processing match #36

Processing match #37

Processing match #38

Processing match #39

Processing match #40

Processing match #41

Processing match #42

Processing match #43

Processing match #44

Processing match #45

Processing match #362

Processing match #363

Processing match #364

Processing match #365

Processing match #366

Processing match #367

Processing match #368

Processing match #369

Processing match #370

Processing match #371

Processing match #372

Processing match #373

Processing match #374

Processing match #375

Processing match #376

Processing match #377

Processing match #378

Processing match #379

Processing match #380

Processing match #381

Processing match #382

Processing match #383

Processing match #384

Processing match #385

Processing match #386

Processing match #387

Processing match #388

Processing match #389

Processing match #390

Processing match #391

Processing match #392

Processing match #393

Processing match #394

Processing match #395

Processing match #396

Processing match #397

Processing match #398

Processing match #399

Processing match #400

Processing match #401

Processing match #402

Processing match #403

Processing match #404

Processing 

Processing match #719

Processing match #720

Processing match #721

Processing match #722

Processing match #723

Processing match #724

Processing match #725

Processing match #726

Processing match #727

Processing match #728

Processing match #729

Processing match #730

Processing match #731

Processing match #732

Processing match #733

Processing match #734

Processing match #735

Processing match #736

Processing match #737

Processing match #738

Processing match #739

Processing match #740

Processing match #741

Processing match #742

Processing match #743

Processing match #744

Processing match #745

Processing match #746

Processing match #747

Processing match #748

Processing match #749

Processing match #750

Processing match #751

Processing match #752

Processing match #753

Processing match #754

Processing match #755

Processing match #756

Processing match #757

Processing match #758

Processing match #759

Processing match #760

Processing match #761

Processing 

Processing match #1074

Processing match #1075

Processing match #1076

Processing match #1077

Processing match #1078

Processing match #1079

Processing match #1080

Processing match #1081

Processing match #1082

Processing match #1083

Processing match #1084

Processing match #1085

Processing match #1086

Processing match #1087

Processing match #1088

Processing match #1089

Processing match #1090

Processing match #1091

Processing match #1092

Processing match #1093

Processing match #1094

Processing match #1095

Processing match #1096

Processing match #1097

Processing match #1098

Processing match #1099

Processing match #1100

Processing match #1101

Processing match #1102

Processing match #1103

Processing match #1104

Processing match #1105

Processing match #1106

Processing match #1107

Processing match #1108

Processing match #1109

Processing match #1110

Processing match #1111

Processing match #1112

Processing match #1113

Processing match #1114

Processing match

Processing match #1416

Processing match #1417

Processing match #1418

Processing match #1419

Processing match #1420

Processing match #1421

Processing match #1422

Processing match #1423

Processing match #1424

Processing match #1425

Processing match #1426

Processing match #1427

Processing match #1428

Processing match #1429

Processing match #1430

Processing match #1431

Processing match #1432

Processing match #1433

Processing match #1434

Processing match #1435

Processing match #1436

Processing match #1437

Processing match #1438

Processing match #1439

Processing match #1440

Processing match #1441

Processing match #1442

Processing match #1443

Processing match #1444

Processing match #1445

Processing match #1446

Processing match #1447

Processing match #1448

Processing match #1449

Processing match #1450

Processing match #1451

Processing match #1452

Processing match #1453

Processing match #1454

Processing match #1455

Processing match #1456

Processing match

...

Processing match #6546

Processing match #6547

Processing match #6548

Processing match #6549

Processing match #6550

Processing match #6551

Processing match #6552

Processing match #6553

Processing match #6554

Processing match #6555

Processing match #6556

Processing match #6557

Processing match #6558

Processing match #6559

Processing match #6560

Processing match #6561

Processing match #6562

Processing match #6563

Processing match #6564

Processing match #6565

Processing match #6566

Processing match #6567

Processing match #6568

Processing match #6569

Processing match #6570

Processing match #6571

Processing match #6572

Processing match #6573

Processing match #6574

Processing match #6575

Processing match #6576

Processing match #6577

Processing match #6578

Processing match #6579

Processing match #6580

Processing match #6581

Processing match #6582

Processing match #6583

Processing match #6584

Processing match #6585

Processing match #6586

Processing match

Processing match #6888

Processing match #6889

Processing match #6890

Processing match #6891

Processing match #6892

Processing match #6893

Processing match #6894

Processing match #6895

Processing match #6896

Processing match #6897

Processing match #6898

Processing match #6899

Processing match #6900

Processing match #6901

Processing match #6902

Processing match #6903

Processing match #6904

Processing match #6905

Processing match #6906

Processing match #6907

Processing match #6908

Processing match #6909

Processing match #6910

Processing match #6911

Processing match #6912

Processing match #6913

Processing match #6914

Processing match #6915

Processing match #6916

Processing match #6917

Processing match #6918

Processing match #6919

Processing match #6920

Processing match #6921

Processing match #6922

Processing match #6923

Processing match #6924

Processing match #6925

Processing match #6926

Processing match #6927

Processing match #6928

Processing match

Processing match #7230

Processing match #7231

Processing match #7232

Processing match #7233

Processing match #7234

Processing match #7235

Processing match #7236

Processing match #7237

Processing match #7238

Processing match #7239

Processing match #7240

Processing match #7241

Processing match #7242

Processing match #7243

Processing match #7244

Processing match #7245

Processing match #7246

Processing match #7247

Processing match #7248

Processing match #7249

Processing match #7250

Processing match #7251

Processing match #7252

Processing match #7253

Processing match #7254

Processing match #7255

Processing match #7256

Processing match #7257

Processing match #7258

Processing match #7259

Processing match #7260

Processing match #7261

Processing match #7262

Processing match #7263

Processing match #7264

Processing match #7265

Processing match #7266

Processing match #7267

Processing match #7268

Processing match #7269

Processing match #7270

Processing match

Processing match #7572

Processing match #7573

Processing match #7574

Processing match #7575

Processing match #7576

Processing match #7577

Processing match #7578

Processing match #7579

Processing match #7580

Processing match #7581

Processing match #7582

Processing match #7583

Processing match #7584

Processing match #7585

Processing match #7586

Processing match #7587

Processing match #7588

Processing match #7589

Processing match #7590

Processing match #7591

Processing match #7592

Processing match #7593

Processing match #7594

Processing match #7595

Processing match #7596

Processing match #7597

Processing match #7598

Processing match #7599

Processing match #7600

Processing match #7601

Processing match #7602

Processing match #7603

Processing match #7604

Processing match #7605

Processing match #7606

Processing match #7607

Processing match #7608

Processing match #7609

Processing match #7610

Processing match #7611

Processing match #7612

Processing match

KeyboardInterrupt: 

### Features para el entrenamiento

Los datos luego serán seleccionados partido a partido, a partir del dataset obtenido. Se realiza la siguiente división:

| Categoría  |  Detalle | 
|---|---|
| ID1 |  ID Jugador 1 |
| ID2 |  ID Jugador 2 |
| RANK |  Ranking ATP |
| POINTS  |  Puntos ATP | 
| FS | Porcentaje  de primeros servicios |
| W1SP | Porcentaje de primeros servicios ganados |
| W2SP | Porcentaje de segundos servicios ganados |
| WSP | Porcentaje de servicios ganados globales |
| WRP | Porcentaje de devolución ganada |
| TPW | Porcentaje de puntos totales ganados |
| TMW | Porcentaje de todos los partidos ganados |
| ACES | Promedio de aces por game |
| DF | Promedio de doble faltas por game |
| UE | Promedio de errores no forzados por game |
| WIS | Promedio de winners por game |
| BP | Porcentaje de break points ganados |
| NA | Porcentaje de puntos en la red ganados |
| A1S | Promedio de velocidad primer servicio |
| A2S | Promedio de velocidad de segundo servicio |
| FATIGUE | Fatiga por partidos jugados en 3 dias anteriores |
| RETIRED | Indica si es el primer partido luego de un retiro |
| DIRECT | Ventaja H2H entre los jugadores |

Cada partido tiene estas características que suelen indicar las diferencias entre ambos jugadores. Por lo que por ejemplo, para ATPRank, si RANK1 es el ranking del jugador 1 y RANK2 es el ranking del jugador 2, RANK = RANK1 - RANK2.
Para el entrenamiento, cada partido tendrá como salida un vector que inidque que jugador gana o que jugador pierde. Es decir, 1 si el jugador ganó o 0 si el jugador perdió.

Finalmente, el dataset será dividido en tres partes:
- Set de entrenamiento (2008 - 2014)
- Set de validación  (2015 - 2016)
- Set de prueba (2017-2018)


In [31]:
results = pd_atp_matches_2017.filter(['winner_name','loser_name'], axis=1)

results['RANK'] = pd_atp_matches_2017['winner_rank'] - pd_atp_matches_2017['loser_rank']
results['POINTS'] = pd_atp_matches_2017['winner_rank_points'] - pd_atp_matches_2017['loser_rank_points']


FS1 = pd_atp_matches_2017['w_1stIn'].div(pd_atp_matches_2017['w_svpt']) 
FS2 =  pd_atp_matches_2017['l_1stIn'].div(pd_atp_matches_2017['l_svpt'])
results['FS'] = FS1 - FS2

W1SP1 = pd_atp_matches_2017['w_1stWon'].div(pd_atp_matches_2017['w_1stIn'])
W1SP2 =  pd_atp_matches_2017['l_1stWon'].div(pd_atp_matches_2017['l_1stIn'])
results['W1SP'] = W1SP1 - W1SP2

W2SP1 = pd_atp_matches_2017['w_2ndWon'].div(pd_atp_matches_2017['w_2ndTot'])
W2SP2 =  pd_atp_matches_2017['l_2ndWon'].div(pd_atp_matches_2017['l_2ndTot'])
results['W2SP'] = W2SP1 - W2SP2

WSP1 = W1SP1 * FS1 + W2SP1 * (1-FS1)
WSP2 = W1SP2 * FS2 + W2SP2 * (1-FS2)
results['WSP'] = WSP1 - WSP2

WRP1=pd_atp_matches_2017['WRP1']
WRP2=pd_atp_matches_2017['WRP2']
results['WRP'] = pd_atp_matches_2017['WRP']

TPW1 = pd_atp_matches_2017['w_pts_w']/pd_atp_matches_2017['total_points']
TPW2 = (pd_atp_matches_2017['total_points']-pd_atp_matches_2017['w_pts_w'])/pd_atp_matches_2017['total_points']
results['TPW'] = TPW1 - (1-TPW1)

results['TMW'] = pd_atp_matches_2017['TMW']

results['ACES'] = (pd_atp_matches_2017['w_ace']-pd_atp_matches_2017['l_ace'])/pd_atp_matches_2017['TotGames']
results['DF'] = (pd_atp_matches_2017['w_df']-pd_atp_matches_2017['l_df'])/pd_atp_matches_2017['TotGames']

BP1 = pd_atp_matches_2017['w_bpSaved']/pd_atp_matches_2017['w_bpFaced']
BP2 = pd_atp_matches_2017['l_bpSaved']/pd_atp_matches_2017['l_bpFaced']
results['BP'] = BP1-BP2

COMPLETE1 = WSP1 * WRP1
COMPLETE2 = WSP2 * WRP2
results['COMPLETE'] = COMPLETE1-COMPLETE2

SERVEADV1 = WSP1 * WRP1
SERVEADV2 = WSP2 * WRP2
results['SERVEADV'] = COMPLETE1-COMPLETE2

#results.rename(columns{'winner_id': 'ID1', 'loser_id': 'ID2'}, inplace=True)

results=results.assign(winner=1)

results.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]


,winner_name,loser_name,RANK,POINTS,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner
0,Grigor Dimitrov,Kei Nishikori,12.0,-2870.0,-0.0348203,0.0537677,0.030000,0.0361378,0.012796,0.054795,1.000000,0.12,0.08,0.314286,0.0228336,0.0228336,1
1,Grigor Dimitrov,Milos Raonic,14.0,-3415.0,0.161673,-0.107143,0.333333,0.120124,0.055238,0.109244,1.000000,0,-0.0952381,NaN,0.0857943,0.0857943,1
2,Kei Nishikori,Stanislas Wawrinka,1.0,-410.0,0.120715,-0.0690154,0.250000,0.0557803,-0.064673,0.086957,1.000000,-0.363636,-0.0454545,0.800000,-0.0206827,-0.0206827,1
3,Milos Raonic,Rafael Nadal,-6.0,2150.0,-0.0870152,0.167107,-0.151553,0.04946,-0.121952,0.071823,1.000000,0.655172,0.103448,0.357143,-0.0602382,-0.0602382,1
4,Grigor Dimitrov,Dominic Thiem,9.0,-1380.0,0.101972,0.0239085,0.014286,0.0451479,0.006865,0.068182,0.333333,-0.107143,-0.0714286,0.357143,0.0230751,0.0230751,1
5,Kei Nishikori,Jordan Thompson,-74.0,4216.0,-0.0369212,0.411654,0.403509,0.397997,0.027894,0.358025,1.000000,0.142857,-0.142857,NaN,0.160301,0.160301,1
9,Dominic Thiem,Samuel Groth,-172.0,3095.0,0.0734748,-0.153247,0.322981,0.0432361,-0.194828,0.073171,1.000000,0,0,0.250000,-0.137956,-0.137956,1
1000,Novak Djokovic,Nicolas Almagro,-74.0,6410.0,0.151564,-0.0893985,0.289803,0.0961202,0.042619,0.083799,1.000000,-0.137931,-0.0344828,0.025974,0.0607026,0.0607026,1
1001,Marius Copil,Guillermo Garcia Lopez,-20.0,70.0,0.116745,0.0277778,0.023109,0.0529186,-0.051798,0.017094,1.000000,0.0277778,-0.111111,-0.100000,-0.0200918,-0.0200918,1
1002,Borna Coric,Mischa Zverev,28.0,-344.0,-0.198871,0.0329268,0.489011,0.151674,-0.001830,0.134752,1.000000,0.181818,-0.136364,0.083333,0.0498485,0.0498485,1


In [57]:
results2 = results.filter(['RANK','FS','W1SP','W2SP','WSP','WRP','TPW','TMW','ACES','DF','BP','COMPLETE','SERVEADV','winner'], axis=1)

results2=results2.assign(player1=' ')
results2=results2.assign(player2=' ')

for i in range(0,len(results2.index)):
    if(results2.loc[i, 'RANK']<0):
        results2.loc[i, 'player1']=results.loc[i,'winner_name']
        results2.loc[i, 'player2']=results.loc[i,'loser_name']
    else:
        results2.loc[i, 'player1']=results.loc[i,'loser_name']
        results2.loc[i, 'player2']=results.loc[i,'winner_name']
        results2.loc[i, 'RANK':'SERVEADV']=results2.loc[i, 'RANK':'SERVEADV']*-1
        results2.loc[i, 'winner']=0

results2=results2.fillna(0)        
results2.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]


,RANK,FS,W1SP,W2SP,WSP,WRP,TPW,TMW,ACES,DF,BP,COMPLETE,SERVEADV,winner,player1,player2
0,-12.0,0.034820,-0.053768,-0.030000,-0.036138,-0.012796,-0.054795,-1.000000,-0.120000,-0.080000,-0.314286,-0.022834,-0.022834,0,Kei Nishikori,Grigor Dimitrov
1,-14.0,-0.161673,0.107143,-0.333333,-0.120124,-0.055238,-0.109244,-1.000000,-0.000000,0.095238,0.000000,-0.085794,-0.085794,0,Milos Raonic,Grigor Dimitrov
2,-1.0,-0.120715,0.069015,-0.250000,-0.055780,0.064673,-0.086957,-1.000000,0.363636,0.045455,-0.800000,0.020683,0.020683,0,Stanislas Wawrinka,Kei Nishikori
3,-6.0,-0.087015,0.167107,-0.151553,0.049460,-0.121952,0.071823,1.000000,0.655172,0.103448,0.357143,-0.060238,-0.060238,1,Milos Raonic,Rafael Nadal
4,-9.0,-0.101972,-0.023909,-0.014286,-0.045148,-0.006865,-0.068182,-0.333333,0.107143,0.071429,-0.357143,-0.023075,-0.023075,0,Dominic Thiem,Grigor Dimitrov
5,-74.0,-0.036921,0.411654,0.403509,0.397997,0.027894,0.358025,1.000000,0.142857,-0.142857,0.000000,0.160301,0.160301,1,Kei Nishikori,Jordan Thompson
9,-172.0,0.073475,-0.153247,0.322981,0.043236,-0.194828,0.073171,1.000000,0.000000,0.000000,0.250000,-0.137956,-0.137956,1,Dominic Thiem,Samuel Groth
1000,-74.0,0.151564,-0.089398,0.289803,0.096120,0.042619,0.083799,1.000000,-0.137931,-0.034483,0.025974,0.060703,0.060703,1,Novak Djokovic,Nicolas Almagro
1001,-20.0,0.116745,0.027778,0.023109,0.052919,-0.051798,0.017094,1.000000,0.027778,-0.111111,-0.100000,-0.020092,-0.020092,1,Marius Copil,Guillermo Garcia Lopez
1002,-28.0,0.198871,-0.032927,-0.489011,-0.151674,0.001830,-0.134752,-1.000000,-0.181818,0.136364,-0.083333,-0.049848,-0.049848,0,Mischa Zverev,Borna Coric



### Soluciones Propuestas
Se propone evaluar si utilizar una red multicapa en el entorno tensorflow para la implementación del proyecto (el cual se entrena con todo el dataset) o una red neuronal recurrente la cual posee una memoria 'selectiva' de partidos anteriores. Estas soluciones están sujetas a cambiar de acuerdo a las nuevas técnicas a aprender en la materia.

### Medición de la Calidad de la Solución Obtenida
La solución obtenida se limitará a predecir el ganador de cada partido. Esto tiene como consecuencia determinar el resultado de cada uno de los torneos a nivel ATP de la temporada, como así también el resultado del ranking al finalizar la temporada. Se contrastaran con los resultados reales, pretendiéndose como mínimo una efectividad superior al 50%.

### Proyectos Similares Implementados en Internet
El proyecto de referencia será la tesis presente en el siguiente link: https://www.doc.ic.ac.uk/teaching/distinguished-projects/2015/m.sipko.pdf el cuál plantea la predicción de partidos de tenis con diversos métodos incluyendo Machine Learning y mostrando sus resultados. Otro proyecto más breve se encuentra en el siguiente link: http://deepakn94.github.io/assets/papers/6.867.pdf. La técnica para solucionar el problema también puede referirse a predictores de otros deportes.


In [58]:
X_train = results2.iloc[:, results2.columns.get_loc('FS'):results2.columns.get_loc('COMPLETE')].values
y_train = results2.iloc[:, results2.columns.get_loc('winner')].values

In [59]:
y_train

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [60]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, kernel_initializer="uniform", input_dim=10, activation="relu")`
  
c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, kernel_initializer="uniform", activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, kernel_initializer="uniform", activation="sigmoid")`
  


In [61]:
# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

c:\users\cufar\anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
2534/2534 [==============================] - 0s 197us/step - loss: 0.5459 - acc: 0.8812
Epoch 2/100
2534/2534 [==============================] - 0s 117us/step - loss: 0.1073 - acc: 0.9882
Epoch 3/100
2534/2534 [==============================] - 0s 129us/step - loss: 0.0434 - acc: 0.9878
Epoch 4/100
2534/2534 [==============================] - 0s 123us/step - loss: 0.0346 - acc: 0.9882
Epoch 5/100
2534/2534 [==============================] - 0s 123us/step - loss: 0.0313 - acc: 0.9886
Epoch 6/100
2534/2534 [==============================] - 0s 129us/step - loss: 0.0297 - acc: 0.9886
Epoch 7/100
2534/2534 [==============================] - 0s 142us/step - loss: 0.0286 - acc: 0.9886
Epoch 8/100
2534/2534 [==============================] - 0s 117us/step - loss: 0.0278 - acc: 0.9890
Epoch 9/100
2534/2534 [==============================] - 0s 136us/step - loss: 0.0274 - acc: 0.9882
Epoch 10/100
2534/2534 [==============================] - 0s 148us/step - loss: 0.0270 - acc: 0.9882

In [62]:
# Predicting the Test set results
y_pred = classifier.predict(X_train)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [63]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_pred)
cm

array([[ 882,   10],
       [  13, 1629]], dtype=int64)